# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [4]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [7]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [8]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [9]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

## Матрица рейтингов

In [10]:
from scipy.sparse import coo_matrix, csr_matrix

In [338]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [339]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [345]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [14]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, нам понадобится вектор для другого пользователя:

In [15]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [16]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [17]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

In [54]:
user_1_rated.toarray()

array([[False, False,  True, ..., False, False, False]])

In [55]:
user_1[user_1_rated.toarray()]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [18]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [19]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [20]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [22]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [110]:
def cosine_similarity_pair_users(u, v):
    ## Your code here
    idx = (u != 0).multiply(v != 0)
    if idx.nnz > 2:
        sim = u[idx].dot(v[idx].T)[0, 0] / (u[idx].dot(u[idx].T)[0, 0]**0.5 * v[idx].dot(v[idx].T)[0, 0]**0.5)
        
        return sim 
    else: 
        return 0

In [111]:
answer1 = round(cosine_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


Введите значение answer1 на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [116]:
cosine_similarity_pair_users(R[0], R[3])

0.9791258160712855

In [139]:
def similar_users(u, R, n_neigbours):
    ## Your code here
    sim_list = []
    for v in R:
        sim_list.append(cosine_similarity_pair_users(R[u], v))
    ind = np.argsort(-np.array(sim_list))[:n_neigbours]
    return ind

In [148]:
np.array2string(similar_users(42, R, 10))

'[  42  281  633  724  815    2  620  650 1692 1506]'

In [140]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [141]:
answer2

'42281633724815262065016921506'

In [147]:
"".join(similar_users(42, R, 10).astype(str))

'42281633724815262065016921506'

Введите значение answer2 без кавычек  на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy. Это будет строка из 29 символов, которая начинается на 42.

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [669]:
def rate_items_user(u, R, n_neigbours, use_rated_only=False):
    """
    use_rated_only: bool, default False
    Флаг о том, используем ли только тех пользователей (ближайших n_neigbours), которые оценили данный конкретный фильм
    или всех  ближайших n_neigbours пользователей, независимо от наличия у них оценки на данный конкретный фильм. 
    
    NOTE!!!: для грейдера проходит только вариант False, то есть всех (by default), несмотря на то, что в лекциях 
    говорилось об использовании только пользователей с оценками
    
    """
    predictions = csr_matrix((1, R.shape[1]))
    ## Your code here
    def get_ind_similar_users(u, R):
        sim_list = []
        for v in R:
            sim_list.append(cosine_similarity_pair_users(R[u], v))
        ind = np.argsort(-np.array(sim_list))
        sim = np.array(sim_list)
        return ind, sim
    
    ind, sim = get_ind_similar_users(u, R)
    ind = ind[1:]  # исключаем самого пользователя (он первый)
    
    for num in tqdm_notebook(range(predictions.shape[1]), total=predictions.shape[1]):
        
        if use_rated_only:
            ind_rated_bool = (R[:, num] != 0).toarray().ravel()  # булевы индексы юзеров, которые оценили фильм "num"
            ind_rated_bool_sorted = ind_rated_bool[ind]
            ind_for_rating_raw = ind[ind_rated_bool_sorted]  # оставляем только пользователей, оценивших фильм "num"
        else:
            ind_for_rating_raw = ind
        
        # берем заданное число ближайших соседей
        ind_for_rating = ind_for_rating_raw[:n_neigbours]
        
        predictions[0, num] = np.sum(sim[ind_for_rating] * R[ind_for_rating, num]) / np.sum(abs(sim[ind_for_rating]))
    
    return predictions

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [670]:
id_ = 20
R_hat = rate_items_user(id_, R, n_neigbours=30)

In [641]:
R_hat_ = R_hat.toarray().ravel()
ind_sorted = np.argsort(-R_hat_)
ind_notwatched = (R[id_].toarray().ravel()[ind_sorted] == 0)  # индексы непросмотренных фильмов
top5 = ind_sorted[ind_notwatched][:5]  # учитываем только непросмотренные ранее фильмы
top5

array([2614,  306,  343, 5573, 6720], dtype=int64)

## Генерация ответа

In [642]:
answer3 = ', '.join(str(i) for i in top5)

In [643]:
answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy Формат ответа - строка вида "X, X, X, X, X", где X - идентификаторы. Вводить ответ следует без кавычек.